# Data Gather

### **library imports**

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import csv
from scipy.signal import butter, filtfilt
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
import pandas as pd
import matplotlib.pyplot as plt
import os  # Add import for os module
from sklearn.utils import resample
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

### **Initialize MediaPipe Pose**

In [3]:
'''This script uses MediaPipe to process video frames for pose estimation.'''
mp_pose = mp.solutions.pose  # Access MediaPipe's pose solution
mp_drawing = mp.solutions.drawing_utils  # Utility for drawing landmarks on frames
pose = mp_pose.Pose(
    static_image_mode=False,  # Process video frames in a continuous stream
    min_detection_confidence=0.5,  # Minimum confidence for detecting landmarks
    min_tracking_confidence=0.5  # Minimum confidence for tracking landmarks
)

### **Function to calculate the angle between two points in the XOY plane**

In [4]:
def angle_from_origin_xoy(p1, p2):
    '''Calculate the angle between two points in the XOY plane.
    Args:
        p1 (tuple): Coordinates of the first point (x1, y1).
        p2 (tuple): Coordinates of the second point (x2, y2).
        Returns:
        float: The angle in degrees between the line connecting the two points and the x-axis.'''
    delta_x = p2[0] - p1[0]
    delta_y = p1[1] - p2[1]  # Invert y-axis for Cartesian coordinates
                
    if delta_x == 0 and delta_y == 0:
        return 0.0  # Handle identical points
                
    angle = np.degrees(np.arctan2(delta_y, delta_x))
    angle = abs(angle)  # Convert -180 to -90 to 90 to 180
    if angle > 90:  # Second and third quadrants
        angle = 180 - angle  # Map 90-180 to 90-0            
    return angle

In [5]:
def draw_angles(frame, origin, target, angle,label):
    # geometry
    cv2.circle(frame, origin, 8, (0,255,0),   -1)
    cv2.circle(frame, target, 8, (0,0,255),   -1)
    cv2.line(  frame, origin, target, (255,255,0), 2)

    # Display metrics text
    x_txt, y_txt = target[0] + 10, max(target[1] - 10, 20)
    cv2.putText(frame, f"{label}: {angle:.1f}", (x_txt, y_txt),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)  # Yellow text

In [6]:
def butter_lowpass_filter(data, cutoff=0.2, order=4):
        b, a = butter(order, cutoff, btype='low', analog=False)
        return filtfilt(b, a, data)

In [7]:
scaler = MinMaxScaler()

### **Function to process video and calculate angles**

In [18]:
from typing import Dict

def process_video_and_calculate_angles(video_path, landmarks: Dict[str,int], classification_class):
    # Initialize an empty DataFrame with VideoName column
    columns = ["Frame", "Time"]
    for landmark_name in landmarks.keys():
        columns.extend([
            f"{landmark_name}_Angle",
            f"{landmark_name}_Velocity",
            f"{landmark_name}_Acceleration"
        ])
    
    processed_data = pd.DataFrame(columns=columns)   
    angles_frames_processed_data = pd.DataFrame(columns=columns)
    temporal_data = pd.DataFrame(columns=columns)  # Initialize temporal_data separately

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video.")
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    scale = (1000, 800)
    frame_now = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or frame is None:
            print("End of video or cannot read the frame.")
            break
        
        print(f"Processing video: {video_path}")
        imagem_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = pose.process(imagem_rgb)

        if result.pose_landmarks:
            mp_drawing.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # Get reference point (midpoint between hips)
            left_hip = result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP]
            right_hip = result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP]
            ref_x = (left_hip.x + right_hip.x) / 2
            ref_y = (left_hip.y + right_hip.y) / 2

            row_data = {
                    "Frame": frame_now,
                    "Time": frame_now / fps
            }

            for landmark_name, landmark_to_compare in landmarks.items():
                # Use the index directly to get the specified landmark
                target_landmark = result.pose_landmarks.landmark[int(landmark_to_compare)]

                # Convert normalized coordinates to image coordinates
                h, w, _ = frame.shape
                ref_pt = (int(ref_x * w), int(ref_y * h))
                target_pt = (int(target_landmark.x * w), int(target_landmark.y * h))

                # Calculate angle
                angule = angle_from_origin_xoy(ref_pt, target_pt)

                if angule is not None:
                    print(f"Frame {frame_now}: {landmark_name} angle: {angule:.2f} degrees")
                    draw_angles(frame, ref_pt, target_pt, angule,landmark_name)                    
                    
                    # Add angle to row_data
                    row_data[f"{landmark_name}_Angle"] = angule
                else:
                    print(f"Frame {frame_now}: Unable to calculate angle.")

            # After processing all landmarks for this frame, append row_data to temporal_data
            temporal_data = pd.concat([temporal_data, pd.DataFrame([row_data])], ignore_index=True)
                
            # Process and show velocity and acceleration for each landmark in the last 30 frames
            if not angles_frames_processed_data.empty:
                for landmark_name in landmarks.keys():
                    last_velocity = angles_frames_processed_data[f"{landmark_name}_Velocity"].dropna().mean()
                    last_acceleration = angles_frames_processed_data[f"{landmark_name}_Acceleration"].dropna().mean()
                                # Improved text formatting with consistent dynamic positioning
                y_offset = 30  # Initial vertical offset
                x_offset = 10  # Horizontal offset for text placement

                for i, landmark_name in enumerate(landmarks.keys()):
                    last_velocity = angles_frames_processed_data[f"{landmark_name}_Velocity"].dropna().iloc[-1]
                    last_acceleration = angles_frames_processed_data[f"{landmark_name}_Acceleration"].dropna().iloc[-1]
                        
                    # Format text
                    text = f"{landmark_name} - Velocity: {last_velocity:.2f} / Acceleration: {last_acceleration:.2f}"
                        
                    # Adjust position dynamically
                    cv2.putText(frame, text, (x_offset, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv2.LINE_AA)
                        
                    y_offset += 30  # Ensure enough spacing between annotations
                                  
            # Every second, append data to angles_processed_data and clear temporal_data
            if (frame_now) % fps == 0:
                # Calculate velocity and acceleration for each landmark angle
                for landmark_name in landmarks.keys():
                    angle_col = f"{landmark_name}_Angle"
                    velocity_col = f"{landmark_name}_Velocity"
                    acceleration_col = f"{landmark_name}_Acceleration"
                    
                    # Apply Butterworth lowpass filter to smooth the angle data
                    temporal_data[f"{angle_col}_filtered"] = butter_lowpass_filter(temporal_data[angle_col].astype(float))
                    
                    # Calculate velocity and acceleration
                    temporal_data[velocity_col] = np.gradient(temporal_data[angle_col], temporal_data["Frame"])
                    temporal_data[acceleration_col] = np.gradient(temporal_data[velocity_col], temporal_data["Frame"])
                    
                    # Normalize velocity and acceleration for each landmark
                    temporal_data[[f"{velocity_col}_normalized", f"{acceleration_col}_normalized"]] = scaler.fit_transform(
                        temporal_data[[velocity_col, acceleration_col]]
                    )
                print(f"Processed angles for frame {frame_now}:")
                # Append the processed temporal data to angles_frames_processed_data
                angles_frames_processed_data = pd.concat([temporal_data], ignore_index=True)
                # Append the processed temporal data to processed_data
                processed_data = pd.concat([processed_data,temporal_data], ignore_index=True)
                temporal_data = temporal_data.iloc[0:0]  # Clear the DataFrame for the next batch                        
                    
        frame_resized = cv2.resize(frame, scale)
        cv2.imshow("Angle Visualization", frame_resized)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_now += 1
        
    cap.release()
    cv2.destroyAllWindows()
    
    processed_data["Classification"] = classification_class
    processed_data["Video Name"] = os.path.basename(video_path)  # Add video name to DataFrame
    
    return processed_data


### **Define the landmarks of interest**

In [9]:
landmarks_ids = {
    "head": 0,  # Head (nose landmark)
    "left_wrist": 15,  # Left wrist
    "right_wrist": 16,  # Right wrist
    "left_ankle": 27,  # Left ankle
    "right_ankle": 28  # Right ankle
}
print("Landmarks de interesse definidos:", landmarks_ids)


Landmarks de interesse definidos: {'head': 0, 'left_wrist': 15, 'right_wrist': 16, 'left_ankle': 27, 'right_ankle': 28}


### **Gather data from videos**

In [ ]:
videos_folder = '.\\Videos\\'
data_folder = '.\\Data\\'
videos = {
    'direct.MOV': 'direct',
    'impro_free.MP4': 'direct',    
    'indirect(cut).mp4': 'indirect',
    'Phrase3.mp4': 'indirect',
    'sustained (direct).MOV': 'direct',
    'sustained (indirect).MOV': 'indirect',
}

def get_videos_data(videos, landmarks, data_folder, videos_folder):
    for video, classification in videos.items():
            video_landmark_processed_data = process_video_and_calculate_angles(
            os.path.join(videos_folder, video), landmarks, classification)
            landmark_processed_data = pd.concat([landmark_processed_data, video_landmark_processed_data], ignore_index=True) if 'landmark_processed_data' in locals() else video_landmark_processed_data
    return landmark_processed_data

landmark_processed_data = get_videos_data(videos, landmarks_ids, data_folder, videos_folder)
landmark_processed_data.to_csv(os.path.join(data_folder, 'landmark_processed_data.csv'), index=False)

Processing video: .\Videos\impro_free.MP4
Processing video: .\Videos\impro_free.MP4
Frame 1: head angle: 89.70 degrees
Frame 1: left_wrist angle: 67.78 degrees
Frame 1: right_wrist angle: 58.11 degrees
Frame 1: left_ankle angle: 79.88 degrees
Frame 1: right_ankle angle: 81.08 degrees
Processing video: .\Videos\impro_free.MP4
Frame 2: head angle: 88.80 degrees
Frame 2: left_wrist angle: 73.80 degrees
Frame 2: right_wrist angle: 65.39 degrees
Frame 2: left_ankle angle: 79.71 degrees
Frame 2: right_ankle angle: 80.42 degrees
Processing video: .\Videos\impro_free.MP4
Frame 3: head angle: 87.01 degrees
Frame 3: left_wrist angle: 86.84 degrees
Frame 3: right_wrist angle: 72.34 degrees
Frame 3: left_ankle angle: 80.76 degrees
Frame 3: right_ankle angle: 79.56 degrees
Processing video: .\Videos\impro_free.MP4
Frame 4: head angle: 83.12 degrees
Frame 4: left_wrist angle: 87.78 degrees
Frame 4: right_wrist angle: 71.93 degrees
Frame 4: left_ankle angle: 81.80 degrees
Frame 4: right_ankle angle: 

C:\Users\asus\AppData\Local\Temp\ipykernel_38832\3318161601.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  temporal_data = pd.concat([temporal_data, pd.DataFrame([row_data])], ignore_index=True)


Frame 5: head angle: 79.72 degrees
Frame 5: left_wrist angle: 80.50 degrees
Frame 5: right_wrist angle: 68.60 degrees
Frame 5: left_ankle angle: 82.49 degrees
Frame 5: right_ankle angle: 75.35 degrees
Processing video: .\Videos\impro_free.MP4
Frame 6: head angle: 75.08 degrees
Frame 6: left_wrist angle: 75.82 degrees
Frame 6: right_wrist angle: 74.58 degrees
Frame 6: left_ankle angle: 85.53 degrees
Frame 6: right_ankle angle: 73.50 degrees
Processing video: .\Videos\impro_free.MP4
Frame 7: head angle: 71.77 degrees
Frame 7: left_wrist angle: 74.01 degrees
Frame 7: right_wrist angle: 71.36 degrees
Frame 7: left_ankle angle: 86.60 degrees
Frame 7: right_ankle angle: 73.09 degrees
Processing video: .\Videos\impro_free.MP4
Processing video: .\Videos\impro_free.MP4
Frame 9: head angle: 74.76 degrees
Frame 9: left_wrist angle: 65.07 degrees
Frame 9: right_wrist angle: 32.41 degrees
Frame 9: left_ankle angle: 79.82 degrees
Frame 9: right_ankle angle: 65.91 degrees
Processing video: .\Videos\i

C:\Users\asus\AppData\Local\Temp\ipykernel_38832\3318161601.py:118: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  processed_data = pd.concat([processed_data,temporal_data], ignore_index=True)


Frame 34: head angle: 83.54 degrees
Frame 34: left_wrist angle: 83.77 degrees
Frame 34: right_wrist angle: 14.25 degrees
Frame 34: left_ankle angle: 72.95 degrees
Frame 34: right_ankle angle: 82.50 degrees
Processing video: .\Videos\impro_free.MP4
Frame 35: head angle: 83.72 degrees
Frame 35: left_wrist angle: 88.49 degrees
Frame 35: right_wrist angle: 32.15 degrees
Frame 35: left_ankle angle: 75.96 degrees
Frame 35: right_ankle angle: 79.36 degrees
Processing video: .\Videos\impro_free.MP4
Frame 36: head angle: 85.49 degrees
Frame 36: left_wrist angle: 88.16 degrees
Frame 36: right_wrist angle: 42.40 degrees
Frame 36: left_ankle angle: 75.55 degrees
Frame 36: right_ankle angle: 71.45 degrees
Processing video: .\Videos\impro_free.MP4
Frame 37: head angle: 87.11 degrees
Frame 37: left_wrist angle: 88.94 degrees
Frame 37: right_wrist angle: 19.18 degrees
Frame 37: left_ankle angle: 75.51 degrees
Frame 37: right_ankle angle: 73.47 degrees
Processing video: .\Videos\impro_free.MP4
Frame 38

C:\Users\asus\AppData\Local\Temp\ipykernel_38832\3318161601.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  temporal_data = pd.concat([temporal_data, pd.DataFrame([row_data])], ignore_index=True)


Frame 4: head angle: 90.00 degrees
Frame 4: left_wrist angle: 69.89 degrees
Frame 4: right_wrist angle: 77.71 degrees
Frame 4: left_ankle angle: 83.51 degrees
Frame 4: right_ankle angle: 78.86 degrees
Processing video: .\Videos\Phrase3.mp4
Frame 5: head angle: 90.00 degrees
Frame 5: left_wrist angle: 70.13 degrees
Frame 5: right_wrist angle: 77.93 degrees
Frame 5: left_ankle angle: 83.76 degrees
Frame 5: right_ankle angle: 78.69 degrees
Processing video: .\Videos\Phrase3.mp4
Frame 6: head angle: 89.80 degrees
Frame 6: left_wrist angle: 70.04 degrees
Frame 6: right_wrist angle: 78.15 degrees
Frame 6: left_ankle angle: 83.81 degrees
Frame 6: right_ankle angle: 78.90 degrees
Processing video: .\Videos\Phrase3.mp4
Frame 7: head angle: 89.39 degrees
Frame 7: left_wrist angle: 70.14 degrees
Frame 7: right_wrist angle: 78.21 degrees
Frame 7: left_ankle angle: 83.83 degrees
Frame 7: right_ankle angle: 78.94 degrees
Processing video: .\Videos\Phrase3.mp4
Frame 8: head angle: 89.19 degrees
Frame

C:\Users\asus\AppData\Local\Temp\ipykernel_38832\3318161601.py:118: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  processed_data = pd.concat([processed_data,temporal_data], ignore_index=True)


Frame 32: head angle: 89.79 degrees
Frame 32: left_wrist angle: 82.08 degrees
Frame 32: right_wrist angle: 86.31 degrees
Frame 32: left_ankle angle: 81.96 degrees
Frame 32: right_ankle angle: 79.80 degrees
Processing video: .\Videos\Phrase3.mp4
Frame 33: head angle: 89.58 degrees
Frame 33: left_wrist angle: 82.58 degrees
Frame 33: right_wrist angle: 86.26 degrees
Frame 33: left_ankle angle: 82.18 degrees
Frame 33: right_ankle angle: 79.80 degrees
Processing video: .\Videos\Phrase3.mp4
Frame 34: head angle: 89.58 degrees
Frame 34: left_wrist angle: 83.85 degrees
Frame 34: right_wrist angle: 86.03 degrees
Frame 34: left_ankle angle: 82.18 degrees
Frame 34: right_ankle angle: 80.02 degrees
Processing video: .\Videos\Phrase3.mp4
Frame 35: head angle: 89.58 degrees
Frame 35: left_wrist angle: 83.75 degrees
Frame 35: right_wrist angle: 85.22 degrees
Frame 35: left_ankle angle: 81.71 degrees
Frame 35: right_ankle angle: 80.02 degrees
Processing video: .\Videos\Phrase3.mp4
Frame 36: head angle

In [22]:
landmark_processed_data.head()

,Frame,Time,head_Angle,head_Velocity,head_Acceleration,left_wrist_Angle,left_wrist_Velocity,left_wrist_Acceleration,right_wrist_Angle,right_wrist_Velocity,...,right_wrist_Velocity_normalized,right_wrist_Acceleration_normalized,left_ankle_Angle_filtered,left_ankle_Velocity_normalized,left_ankle_Acceleration_normalized,right_ankle_Angle_filtered,right_ankle_Velocity_normalized,right_ankle_Acceleration_normalized,Classification,Video Name
0,1,0.034483,89.596515,0.202448,-0.000705,69.863697,0.203767,-0.120883,77.157246,0.019489,...,0.145172,0.933033,82.849592,0.835104,0.788211,79.164269,0.129473,0.459212,indirect,Phrase3.mp4
1,2,0.068966,89.798963,0.201742,-0.050965,70.067463,0.082884,-0.145924,77.176735,0.202603,...,0.302245,0.835488,83.096864,0.944829,0.750313,79.035187,0.178955,0.547785,indirect,Phrase3.mp4
2,3,0.103448,90.000000,0.100518,-0.100871,70.029464,-0.088082,-0.016437,77.562452,0.267471,...,0.357889,0.617074,83.326555,1.000000,0.653356,78.926534,0.376012,0.490789,indirect,Phrase3.mp4
3,4,0.137931,90.000000,0.000000,-0.100874,69.891300,0.050010,0.081674,77.711677,0.185818,...,0.287848,0.592555,83.523324,0.970152,0.493238,78.853720,0.330529,0.516820,indirect,Phrase3.mp4
4,5,0.172414,90.000000,-0.101229,-0.152377,70.129483,0.075266,-0.022128,77.934088,0.220965,...,0.317996,0.591230,83.675338,0.794827,0.357994,78.822561,0.570958,0.688210,indirect,Phrase3.mp4


In [30]:
# Group by each second (using integer part of 'Time') and aggregate min, max, mean for all columns except 'Classification', 'Video Name', and 'Frame'
grouped_stats = landmark_processed_data.groupby(landmark_processed_data['Time'].astype(int)).agg(
    {col: ['min', 'max', 'mean'] for col in landmark_processed_data.columns if col not in ['Classification', 'Video Name', 'Frame', 'Time']}
)

# Optionally, reset index for a flat DataFrame
grouped_stats = grouped_stats.reset_index()

grouped_stats.head(30)

Time head_Angle                       head_Velocity                       \
               min        max       mean           min        max      mean   
0     0  88.152390  90.000000  88.919953     -0.407890   0.504312  0.007180   
1     1  78.558956  90.000000  87.060401     -1.432399   0.607353 -0.404344   
2     2  69.443955  88.115950  76.011321     -1.984765   1.913097  0.369683   
3     3  69.274441  89.660976  83.701568     -6.942960   7.193437 -0.398744   
4     4  30.579227  90.000000  66.709602    -21.069077   7.577621 -1.009126   
5     5  21.801409  85.601295  50.872234    -30.472698  13.632764 -1.281855   
6     6  50.659482  69.489141  65.266073     -0.609465   4.536179  0.696707   
7     7  68.270210  83.271488  75.629307     -0.994556   1.287345  0.510610   
8     8  80.630615  89.716360  84.368865     -1.009184   1.359051 -0.108550   
9     9  81.792784  90.000000  86.716261     -0.122426   0.742983  0.283090   
10   10  87.492298  90.000000  89.339876     -0.626798   0.245903 -0.066819   
11   11  85.634742  87.887589  86.387104     -0.426717   0.356613  0.024272   
12   12  81.702855  89.714949  87.268781     -1.402410   0.385486 -0.205270   
13   13  73.855661  84.999355  79.977049     -1.479716   1.378995  0.109737   
14   14  63.434949  89.104826  77.068360    -12.419415   9.217474 -1.399619   
15   16  72.758541  83.290163  76.575956      0.728021   4.897262  2.590858   
16   17  82.234834  82.234834  82.234834     -1.055329  -1.055329 -1.055329   

   head_Acceleration                       ...  \
                 min        max      mean  ...   
0          -0.157271   0.256128  0.012608  ...   
1          -0.694088   0.352220 -0.044576  ...   
2          -0.615726   0.667698 -0.015635  ...   
3          -4.885362   7.068198  0.159522  ...   
4         -11.651328   4.660732 -1.061592  ...   
5         -15.011384  16.199009 -0.965244  ...   
6          -0.694029   0.481535 -0.142494  ...   
7          -0.424363   0.576224 -0.010820  ...   
8          -0.656817   0.589031  0.036125  ...   
9          -0.250104   0.269430  0.000988  ...   
10         -0.213358   0.212852  0.007154  ...   
11         -0.422130   0.201213 -0.008667  ...   
12         -0.694133   0.825247 -0.013820  ...   
13         -0.722082   0.710106  0.046491  ...   
14         -6.095494   4.458029 -0.804204  ...   
15         -2.976296   2.084621  0.183434  ...   
16         -3.793006  -3.793006 -3.793006  ...   

   left_ankle_Acceleration_normalized right_ankle_Angle_filtered             \
                                 mean                        min        max   
0                            0.609205                  78.606578  79.164269   
1                            0.534077                  79.077967  82.324973   
2                            0.536828                  59.885155  82.288132   
3                            0.478043                  24.971375  95.210078   
4                            0.455509                   9.744890  29.723941   
5                            0.484495                  20.536238  61.728747   
6                            0.505371                  50.547509  66.577612   
7                            0.599135                  67.020855  86.363998   
8                            0.592017                  85.623530  88.923714   
9                            0.348755                  78.406333  85.046453   
10                           0.644153                  80.262623  85.645913   
11                           0.442463                  79.379727  82.293107   
12                           0.621786                  77.541667  79.515855   
13                           0.413289                  35.324736  77.407690   
14                           0.599378                  40.230259  90.128935   
15                           0.641377                  38.837703  48.274634   
16                           0.677763                  53.073982  53.073982   

              right_ankle_Velocity_normalized   

In [ ]:
# Group by each second
landmark_grouped_processed_data = landmark_processed_data.groupby(landmark_processed_data['Time'].astype(int))
landmark_grouped_processed_data = landmark_grouped_processed_data.agg({
            'Filtered Angle (degrees)': ['std', 'mean', 'min', 'max'],
            'velocity': ['std', 'mean', 'min', 'max'],
            'acceleration': ['std', 'mean', 'min', 'max']
        }).reset_index()
    # Flatten MultiIndex columns
landmark_grouped_processed_data.columns = ['Time (seconds)',
                        'angle_std', 'angle_mean', 'angle_min', 'angle_max',
                        'velocity_std', 'velocity_mean',  'velocity_min', 'velocity_max',
                        'acceleration_std', 'acceleration_mean', 'acceleration_min', 'acceleration_max']
landmark_grouped_processed_data.to_csv(os.path.join(data_folder, 'all_landmarks_data_Processed_combined.csv'), index=False)

KeyError: "Column(s) ['Filtered Angle (degrees)', 'acceleration', 'velocity'] do not exist"

### **Print DataFrame statistics**

In [ ]:
landmark_processed_data.describe(include='all')

,Time (seconds),Video Name,Classification,angle_std_head,angle_medium_head,angle_median_head,angle_min_head,angle_max_head,velocity_std_head,velocity_medium_head,...,velocity_std_right_ankle,velocity_medium_right_ankle,velocity_median_right_ankle,velocity_min_right_ankle,velocity_max_right_ankle,acceleration_std_right_ankle,acceleration_medium_right_ankle,acceleration_median_right_ankle,acceleration_min_right_ankle,acceleration_max_right_ankle
count,24.000000,24,24,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,...,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000
unique,NaN,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Phrase3.mp4,indirect,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,16,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,6.208333,NaN,NaN,6.949454,76.054129,76.617481,64.868327,85.695832,1.185817,-0.167940,...,1.793486,-0.473514,-0.603870,-3.392260,2.788511,0.645593,0.022661,0.011727,-1.111216,1.146996
std,4.596588,NaN,NaN,6.830514,13.972433,14.960603,22.647242,5.106455,1.236374,1.235671,...,1.895642,1.728296,1.748803,3.285879,4.399260,0.681255,0.434993,0.444971,1.315427,1.373732
min,0.000000,NaN,NaN,0.582127,40.900191,32.344184,16.187134,71.978330,0.098639,-2.831424,...,0.145687,-6.294967,-6.471038,-12.586660,-5.708411,0.035762,-0.594921,-0.561045,-4.416133,-0.234414
25%,2.750000,NaN,NaN,2.022171,72.245347,74.197294,60.644802,82.056637,0.326264,-0.468678,...,0.353615,-0.422123,-0.777490,-4.751074,0.384817,0.108259,-0.117442,-0.076093,-2.072895,0.157304
50%,5.500000,NaN,NaN,5.131147,80.550428,81.412039,73.433874,88.645806,0.854609,-0.000666,...,1.450209,-0.101117,0.000029,-2.568866,1.516738,0.418743,-0.011002,-0.015382,-0.711759,0.700782
75%,9.250000,NaN,NaN,7.487571,86.375945,87.382549,77.821015,89.306147,1.802350,0.229123,...,2.237565,0.296474,0.268928,-0.674596,4.266979,1.037263,0.012648,0.022313,-0.142418,1.745160


In [ ]:
landmark_processed_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 78 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Time (seconds)                   24 non-null     int32  
 1   Video Name                       24 non-null     object 
 2   Classification                   24 non-null     object 
 3   angle_std_head                   24 non-null     float64
 4   angle_medium_head                24 non-null     float64
 5   angle_median_head                24 non-null     float64
 6   angle_min_head                   24 non-null     float64
 7   angle_max_head                   24 non-null     float64
 8   velocity_std_head                24 non-null     float64
 9   velocity_medium_head             24 non-null     float64
 10  velocity_median_head             24 non-null     float64
 11  velocity_min_head                24 non-null     float64
 12  velocity_max_head       